In [1]:
# %pip install -U crewai
# %pip install -U langchain-groq 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 760.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 191.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 38.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 86.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 197.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 212.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 559.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 533.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 534.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━

In [1]:
import os
from getpass import getpass

from crewai import Agent,Task,Crew,Process
from crewai_tools.tools import SerperDevTool
from langchain_groq import ChatGroq

In [2]:
SERPER_API_KEY = "9e197ad3ca682f2775833b0f274a1f66a6b563e7"
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
GROQ_API_KEY = "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"
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [3]:
llm = ChatGroq(model="mixtral-8x7b-32768",groq_api_key=GROQ_API_KEY)
print(llm.invoke("hi"))

content="Hello! How can I help you today? If you have any questions about a specific topic, feel free to ask. I'll do my best to provide a clear and helpful response." response_metadata={'token_usage': {'completion_time': 0.057874673, 'completion_tokens': 38, 'prompt_time': 0.003350769, 'prompt_tokens': 11, 'queue_time': None, 'total_time': 0.061225442000000005, 'total_tokens': 49}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-e926a611-9929-4e3c-808a-5b90ba9f5063-0'


In [4]:
search_tool = SerperDevTool(n_results=5)

In [13]:
researcher = Agent(
    role = "Researcher",
    goal='Find accurate and up-to-date medical information from reputable sources to assist users with their health-related queries, based on this user query: {subject_area}. Also Provide concise and accurate summaries of collected medical information.',
    backstory=(
    "As a dedicated health information searcher, your primary mission is to "
    "scour the web for reliable and precise medical information. With a keen eye for credible sources "
    "and a passion for accuracy, you aim to provide the most relevant data to help users make informed health decisions, along with potential treatment options and medications that may be suitable for their condition."
    "Apart from that, you have a detail-oriented summarizer with a talent for condensing complex medical information "
    "into clear and accessible summaries. Your work ensures that users receive accurate and understandable information."
    ),
    llm = llm,
    max_iter = 5,
    tools = [search_tool],
    allow_delegation=True,
    verbose=True
)


In [14]:
research_task = Task(
    description = (
    "Find accurate and up-to-date medical information from reputable sources to assist users with their health-related queries, based on this user query: {subject_area}"
    "Set the input parameter as : search_query'"
  ),
    expected_output='A brief summary of the latest advancements in {subject_area}.',
    tools = [search_tool],
    agent = researcher
)


In [15]:
%time

crew = Crew(
    agents = [researcher],
    tasks = [research_task],
    max_rpm = 3,
    cache=False
)

result = crew.kickoff(inputs={'subject_area':"I ate an icecream and have a headache"})
print(result)

2024-06-21 22:30:48,471 - 8637893632 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs


> Entering new CrewAgentExecutor chain...
The user is experiencing a headache after eating ice cream, which could be related to various factors such as brain freeze or an ice cream headache, food allergies, or underlying health conditions. To provide accurate and up-to-date medical information, I should search the internet for reputable sources discussing these possibilities.

Action: Search the internet
Action Input: {'search_query': 'brain freeze vs ice cream headache vs food allergies vs headache after eating ice cream'}
 


Search results: Title: Ice Cream Headaches & Brain Freezes - Why They Happen - WebMD
Link: https://www.webmd.com/migraines-headaches/ice-cream-headache
Snippet: But brain freeze is generally thought to be harmless, so that triple-scoop cone won't trigger a migraine or any other type of serious headache.
---
Title: Brain freeze: The science behind ice cream headache
Link: https://www.medica

In [16]:
from IPython.display import Markdown
Markdown(result)


A headache after eating ice cream is typically known as a brain freeze or "ice cream headache." This type of headache is caused by a cold stimulus to the roof of the mouth, resulting in blood vessels constricting and then dilating. It is generally harmless and can be prevented by eating cold foods more slowly.